# Investigating a simple statistical model

In [ ]:
import pyhf
import cabinetry

cabinetry.set_logging()

In [ ]:
spec = {
    "channels": [
        {
            "name": "SR",
            "samples": [
                {
                    "data": [3.0, 10.0],
                    "modifiers": [
                        {"data": None, "name": "mu", "type": "normfactor"},
                    ],
                    "name": "Signal"
                },
                {
                    "data": [50.0, 30.0],
                    "modifiers": [
                        {
                            "data": {"hi": 1.05, "lo": 0.95},
                            "name": "normalization_uncertainty",
                            "type": "normsys"
                        },
                        {
                            "data": {"hi_data": [55.0, 35.0], "lo_data": [45.0, 25.0]},
                            "name": "shape_uncertainty",
                            "type": "histosys"
                        }
                    ],
                    "name": "Background"
                }
            ]
        }
    ],
    "measurements": [{"config": {"parameters": [], "poi": ""}, "name": "example"}],
    "observations": [{"data": [62.0, 45.0], "name": "SR"}],
    "version": "1.0.0"
}

ws = pyhf.Workspace(spec)

model = ws.model()
data = ws.data(model)

In [ ]:
# the model has parameters
par_values = model.config.suggested_init()
for idx, par_name in enumerate(model.config.par_names):
    print(f"{par_name}: {par_values[idx]}")

In [ ]:
# we can evaluate the model prediction for any parameter setting
model.expected_data(par_values)

In [ ]:
# or get the -2*NLL directly
-2 * model.logpdf(par_values, data)

In [ ]:
# or plot the prediction
model, data = cabinetry.model_utils.model_and_data(ws)
pred = cabinetry.model_utils.prediction(model)
_ = cabinetry.visualize.data_mc(pred, data)

# What does a nuisance parameter do?

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.stats import poisson


# Gaussian for constraint term
x = np.arange(-3, 3, 0.01)
a = 0  # auxiliary data
gauss = (
    lambda a, theta: 1 / (2 * np.pi) ** 0.5 * np.exp(-0.5 * (a - theta) ** 2 / 1**2)
)

# Poisson for observed counts
x_p = np.arange(110, 170, 1)

In [ ]:
spec = {
    "channels": [
        {
            "name": "Signal_region",
            "samples": [
                {
                    "data": [140.0],
                    "modifiers": [
                        {
                            "data": {"hi": 1.08, "lo": 0.92},
                            "name": "luminosity",
                            "type": "normsys",
                        }
                    ],
                    "name": "Dummy",
                }
            ],
        }
    ],
    "measurements": [
        {
            "config": {"parameters": [], "poi": ""},
            "name": "Luminosity uncertainty example",
        }
    ],
    "observations": [{"data": [140.0], "name": "Signal_region"}],
    "version": "1.0.0",
}

ws = pyhf.Workspace(spec)
model = ws.model()

What effect do changes of a parameter of the model (a nuisance parameter in this case) have?

In [ ]:
np_value = 0.0  # NP value, change this to study the effect!

fig, ax = plt.subplot_mosaic("AB\nCC\nDD", tight_layout=True, figsize=(6, 8))
ax["A"].plot(np_value, 0, marker="X", markersize=15)
ax["A"].set_xlabel("nuisance parameter value")
ax["A"].set_xlim([-2.5, 2.5])
ax["A"].set_xticks([-2, -1, 0, 1, 2])
ax["A"].set_yticks([])
ax["A"].set_title(f"$\\theta=${np_value:.2f}")

model_yield = model.expected_actualdata([np_value])[0]
ax["B"].bar(0, model_yield, label="prediction")
ax["B"].plot(0, 148, "X", markersize=10, label="observed data", c="C2")
ax["B"].set_ylabel("yield")
ax["B"].set_ylim([0, 250])
ax["B"].set_xticks([])
ax["B"].set_title(f"{model_yield:.2f} events")
ax["B"].legend(frameon=False, loc="upper right")

n_obs = 148
arrow_base = poisson.pmf(n_obs, model_yield)
ax["C"].plot(x_p, poisson.pmf(x_p, model_yield))
ax["C"].plot([model_yield, model_yield], [0, poisson.pmf(round(model_yield), model_yield)], "--", c="grey")
ax["C"].annotate(
    "evaluated here",
    xy=(148, arrow_base + 0.005),
    xytext=(148, arrow_base + 0.02),
    arrowprops=dict(facecolor="black", shrink=0.05),
    ha="center",
)
ax["C"].set_title(
    f"Pois($n=${n_obs} | $\\nu(\\theta)=${model_yield:.2f}) = {poisson.pmf(n_obs, model_yield):.3f}"
)
ax["C"].set_xlabel("observed events $n$")
ax["C"].set_ylabel("likelihood")
ax["C"].set_xlim([110, 170])
ax["C"].set_ylim([0, 0.06])

arrow_base = gauss(0, np_value)
ax["D"].plot(x, gauss(0, x - np_value), linewidth=2)
ax["D"].plot([np_value, np_value], [0, gauss(0, 0)], "--", c="grey")
ax["D"].annotate(
    "evaluated here",
    xy=(0, arrow_base + 0.02),
    xytext=(0, arrow_base + 0.15),
    arrowprops=dict(facecolor="black", shrink=0.05),
    ha="center",
)
ax["D"].set_title(f"constraint term Gauss($a=0$ | $\\theta=${np_value:.2f}) = {gauss(0, np_value):.2f}")
ax["D"].set_xlabel("auxiliary data $a$")
ax["D"].set_ylabel("likelihood")
ax["D"].set_xlim([-3, 3])
ax["D"].set_ylim([0, 0.6]);

In [ ]:
# what is the best-fit value here in practice?
print(f"best-fit NP value: {pyhf.infer.mle.fit([n_obs, a], model)[0]:.2f}")

# A quick tour through `cabinetry`

In [ ]:
import copy
import glob
import json
import pathlib
import shutil

import boost_histogram as bh
from dask.distributed import Client, LocalCluster, wait
import hist
import numpy as np
from pyhf.contrib.utils import download

<br>

We will have a look at three different ways of using `cabinetry` in this notebook:
- creating a statistical model,
- performing statistical inference with our model,
- exploring the statistical model of an ATLAS analysis.

There is also bonus material at the end for you to look through.

<br>

# Creating a statistical model
Statistical models are constructed following instructions in a configuration. You can specify your model in a configuration in `YAML` or `JSON` format, or alternatively as a Python dictionary. Here we will go the dictionary route. Have a look at the `config_example.yml` file for a `YAML` example.

<br>

### Defining the model

There are a few things we need. First up, some general settings:

In [ ]:
config = {
   "General":{
      "Measurement": "minimal_example",
      "POI": "Signal_norm",              # parameter of interest, which we want to measure 
      "InputPath": "input/{SamplePath}", # where to find input data
      "HistogramFolder": "histograms/"

   }
}

Note the `input/{SamplePaths}` structure, which we will get back to shortly.

Now it is time to think more about physics, let's define a phase space region that contains data we want to fit to.

In [ ]:
config.update({
   "Regions":[
      {
         "Name": "Signal_region",
         "Filter": "lep_charge > 0",           # event selection 
         "Variable": "jet_pt",                 # which variable we bin histograms in
         "Binning": [200, 300, 400, 500, 600]
      }
   ]
})

`"Regions"` is a list, because we can use events from more than one phase space region.

We also need to specify where our observed data is stored. Considerations about physics become important now: which types of processes do we expect to show up in this phase space region? We list them below, those samples are typically simulated with Monte Carlo methods.

In [ ]:
config.update({
   "Samples":[
      {
         "Name": "Data",
         "Tree": "pseudodata",
         "SamplePath": "data.root",
         "Data": True                       # observed data is handled differently, need to distinguish
      },
      {
         "Name": "Signal",
         "Tree": "signal",
         "SamplePath": "prediction.root",
         "Weight": "weight"                 # weights: Monte Carlo integration, simulation correction etc.
      },
      {
         "Name": "Background",
         "Tree": "background",
         "SamplePath": "prediction.root",
         "Weight": "weight"
      }
   ]
})

`"Samples"` is again a list, allowing us to include arbitrarly many different types of processes we may need to consider. The `"SamplePath"` option replaces the placeholder in `input/{SamplePath}`, and specifies where each sample can be found (see the [documentation](https://cabinetry.readthedocs.io/en/latest/core.html#input-file-path-specification-for-ntuples)).

We are almost done defining our statistical model. What about systematic uncertainties? For now, we won't define any.

In [ ]:
config.update({"Systematics": []})

<br>

There is one more thing we need though: we said our POI (parameter of interest) was `"Signal_norm"`. That is a normalization factor, describing the normalization of somthing. We still need to define what it does!

In [ ]:
config.update({
   "NormFactors":[
      {
         "Name": "Signal_norm",
         "Samples": "Signal",    # we want this parameter to scale the signal
         "Nominal": 1,
         "Bounds": [-5, 10]
      }
   ]
})

<br>

It is a good idea to validate that our configuration satisfies the `cabinetry` configuration schema.

In [ ]:
cabinetry.configuration.validate(config)

<br>

That looks good, but is not sufficient to be sure that everything is fine (some things will only show up at runtime). If we use some settings `cabinetry` does not know, or forget some it expects, we would see an error. We can also get some summary information about the configuration we defined.

In [ ]:
cabinetry.configuration.print_overview(config)

<br>

### Creating histograms

We now need to create the required histograms for our statistical model. `cabinetry` implements the logic to figure out which histograms are needed, to create all instructions, and to send those off for execution. You can view our `config` as a generic specification and `cabinetry` as the reference implementation handling it, but you could just as well use your own code to interact with the specification.

In [ ]:
cabinetry.templates.build(config, method="uproot")

`cabinetry` used `uproot` and `boost-histogram` to create three histograms for us: the distribution of our three samples in the one phase space region we defined. We also see a warning: there are no expected signal events in the first bin of the histogram.

The histograms are saved to the folder specified under HistogramFolder in the General settings in the configuration file. In this case, this folder is `histograms/`:

In [ ]:
glob.glob("histograms/*")

#### Post-processing and visualization

A post-processing step can be run to apply optional operations like histogram smoothing. See the bonus material.

In [ ]:
cabinetry.templates.postprocess(config)

You can also provide existing histograms you built yourself for `cabinetry` to use, see the [cabinetry-tutorials](https://github.com/cabinetry/cabinetry-tutorials) repository for an example.

<br>
Let's visualize the histograms we have produced.

In [ ]:
_ = cabinetry.visualize.data_mc_from_histograms(config)

We see our expected distributions of signal and background, as well as our observed (pseudo-) data.

<br>

### A more complex model: adding systematic uncertainties

Let's make our model a bit more realistic and add a few systematic uncertainties:
- a 5% luminosity uncertainty,
- a `Modeling` uncertainty derived from comparing our nominal background prediction to that of a different simulator,
- a `WeightBasedModeling` modeling uncertainty derived from varying the weights we apply to background events.

In [ ]:
config.update({
   "Systematics":[
      {
         "Name": "Luminosity",
         "Up": {"Normalization": 0.05},
         "Down": {"Normalization": -0.05},
         "Type": "Normalization"
      },
      {
         "Name":"Modeling",
         "Up": {"Tree": "background_varied"},
         "Down": {"Symmetrize": True},
         "Samples": "Background",
         "Type": "NormPlusShape"
      },
      {
         "Name": "WeightBasedModeling",
         "Up": {"Weight": "weight_up"},
         "Down": {"Weight": "0.7*weight"},
         "Samples": "Background",
         "Type": "NormPlusShape"
      }
   ],
})

<br>
These new systematic uncertainties require new histograms, so let's build those.

In [ ]:
cabinetry.templates.build(config, method="uproot")
cabinetry.templates.postprocess(config)

<br>
We can visualize the template histograms corresponding to systematic variations of our model.

In [ ]:
_ = cabinetry.visualize.templates(config)

The top figure is our `Modeling` uncertainty, where we compare our nominal background prediction to that of another simulator, here called the "up" variation. At the bottom are the weight-based variations: you can see the "down" variation, defined by multiplying the nominal weight by `0.7`, is a factor `0.7` smaller than nominal.

<br>

### Building a workspace

With all relevant histograms available, we can now construct a `pyhf` workspace. This is our serialized fit model. It contains everything needed to construct a likelihood function via `pyhf`, which is then used for inference.

In [ ]:
workspace_path = "example_workspace.json"
spec = cabinetry.workspace.build(config)
cabinetry.workspace.save(spec, workspace_path)

The `spec` object is a dictionary holding our workspace specification.

In [ ]:
print(json.dumps(spec, sort_keys=True, indent=4))

Saving is optional, we could directly continue with the `spec` object. `pyhf` has validated our workspace, and we are now ready for statistical inference.

<br>

### Model structure

It can be helpful to visualize the modifier structure of the statistical model we have built to catch potential issues. The `visualize.modifier_grid` function creates a figure showcasing the information about which modifiers (indicated by color) act on which region and sample when a given parameter (on the horizontal axis) is varied. To split this visualization from one table per region to one table per sample, use `split_by_sample=True`.

We need the fit model (containing the probability density function) for the visualization, which we create from the workspace specification. We directly use the `pyhf` API for this here, in the next section you will see the `cabinetry` API used for the same task.

In [ ]:
cabinetry.visualize.modifier_grid(pyhf.Workspace(spec).model())

<br>

# Performing statistical inference with our model

To perform inference, we need two things: a probability density function (pdf), or `model`, and data to fit it to. Both are derived from the workspace specification.

In [ ]:
model, data = cabinetry.model_utils.model_and_data(spec)

You can see our `Signal_norm` normalization showing up, in addition to parameters for the systematic uncertainties we defined.

There is also `staterror_Signal_region`: these are parameters automatically created by `cabinetry` to encode systematic uncertainty due to the finite sample size of our predicted distributions for signal and background.

`data` is a list, starting with the observed counts per bin.

In [ ]:
data

What comes after is so-called auxiliary data. Check out [this `pyhf` tutorial](https://pyhf.github.io/pyhf-tutorial/HelloWorld.html#auxiliary-data) to learn more about this important concept. You can view it as data observed in previous measurements, which inform our current model. In this case the auxiliary data is associated to the `staterror_Signal_region` parameter. 

<br>

### Maximum likelihood estimate (MLE)

Let's fit our model to data to obtain the maximum likelihood estimate (MLE).

In [ ]:
fit_results = cabinetry.fit.fit(model, data)

<br>
The fit converged, and we see the best-fit parameter results reported. The results are stored in a named tuple. This allows for easy access of the results.

In [ ]:
for label, result, unc in zip(fit_results.labels, fit_results.bestfit, fit_results.uncertainty):
    print(f"{label}: {result:.3f} +/- {unc:.3f}")

<br>

It is helpful to visualize the fit results. Let's start with the *pull plot* showing us best-fit parameter results.

In [ ]:
cabinetry.visualize.pulls(fit_results, exclude="Signal_norm")

<br>

The parameter correlation matrix has a handy `pruning_threshold` setting to filter out parameters that are not highly correlated with others.

In [ ]:
cabinetry.visualize.correlation_matrix(fit_results, pruning_threshold=0.1)

<br>
How does the model look like after fit to data? Let's first look again at the model before the fit to data. We use information from the workspace for plotting, and from the config for cosmetics (axis label / binning). The results include the effect of systematic uncertainties.

In [ ]:
model_pred = cabinetry.model_utils.prediction(model)
figures = cabinetry.visualize.data_mc(model_pred, data, config=config)

It is possible to edit the figures created by `cabinetry` using the `matplotlib` API. The example below modifies an axis label to use $\LaTeX$. `cabinetry` version 0.5.2 also added the possibility to more conveniently change the colors of histograms in the stack as shown below.

In [ ]:
figures = cabinetry.visualize.data_mc(model_pred, data, config=config,
                                      colors={"Signal": "tomato", "Background": "navajowhite"}, close_figure=True)
ratio_panel = figures[0]["figure"].get_axes()[1]
ratio_panel.set_xlabel("jet $p_T$")
figures[0]["figure"]  # show figure

Yield tables can also be created from a model prediction, and compared to data. Optional keyword arguments control whether yields per bin are shown (`per_bin=True`, default) and whether bins summed per region are shown (`per_channel=True`, disabled by default). The yield table is also saved to disk by default, in a format customizable via the `table_format` argument.

In [ ]:
_ = cabinetry.tabulate.yields(model_pred, data)

This table was also saved to disk.

In [ ]:
!cat tables/yields_per_bin_pre-fit.txt

<br>

We create the post-fit version of this plot by passing in `fit_results`.

In [ ]:
model_pred_postfit = cabinetry.model_utils.prediction(model, fit_results=fit_results)
_ = cabinetry.visualize.data_mc(model_pred_postfit, data, config=config)

The contribution of signal in green has increased, consistent with the normalization `Signal_norm` having been fit to a value greater than `1`.

<br>

### Expected sensitivity with the Asimov dataset

We can evaluate the expected performance of our model with the so-called Asimov dataset (see [arXiv:1007.1727](https://arxiv.org/abs/1007.1727)).

In [ ]:
asimov_data = cabinetry.model_utils.asimov_data(model)
_ = cabinetry.fit.fit(model, asimov_data)

By definition, none of the parameters are pulled away from their initial values in this fit.

<br>

### Beyond MLEs: discovery significance and parameter limits

Now that we ran a basic fit, let's do something slightly more involved: calculate discovery significance.

In [ ]:
significance_results = cabinetry.fit.significance(model, data)

The results are again packaged up in a named tuple. The observed significance is higher than the expected significance, consistent with fitting a `Signal_norm` value above one.

We can also calculate expected and observed 95% confidence level upper parameter limits with the [CLs method](https://doi.org/10.1088%2F0954-3899%2F28%2F10%2F313). The implementation uses Brent bracketing to find `CLs=0.05` crossings. Let's use slightly different data for this: it is simple to switch out!

In [ ]:
data_limit = [112, 129, 92, 63] + model.config.auxdata  # need auxiliary data as well
limit_results = cabinetry.fit.limit(model, data_limit)
cabinetry.visualize.limit(limit_results)

The observed limit follows the expectation quite closely with this new data we have used, which is very similar to the background-only prediction.

<br>

# Exploring the statistical model of an ATLAS analysis

Let's explore the statistical model used in an ATLAS search for electroweakinos: [Eur.Phys.J.C 80 (2020) 8, 691](https://inspirehep.net/literature/1755298). The corresponding likelihoods are available on HEPData: [doi:10.17182/hepdata.90607.v2](https://doi.org/10.17182/hepdata.90607.v2).

We can download it, pick one of the available signals with `pyhf`, and are ready to go.

In [ ]:
download("https://www.hepdata.net/record/resource/1267798?view=true", "electroweakinos")
ATLAS_ws = pyhf.Workspace(json.load(open("electroweakinos/1Lbb-likelihoods-hepdata/BkgOnly.json")))
patchset = pyhf.PatchSet(json.load(open("electroweakinos/1Lbb-likelihoods-hepdata/patchset.json")))
ATLAS_ws = patchset.apply(ATLAS_ws, "C1N2_Wh_hbb_700_400")
cabinetry.workspace.save(ATLAS_ws, "electroweakinos.json")

<br>`pyhf` has a helpful command line interface utility to summarize the workspace content.

In [ ]:
!pyhf inspect electroweakinos.json | head -n 6

<br>

Let's extract model and data with `cabinetry`.

In [ ]:
ATLAS_model, ATLAS_data = cabinetry.model_utils.model_and_data(ATLAS_ws)

<br>
We are ready to take a closer look at the content. Which phase space regions are included?

In [ ]:
ATLAS_model_pred = cabinetry.model_utils.prediction(ATLAS_model)
_ = cabinetry.visualize.data_mc(ATLAS_model_pred, ATLAS_data)

<br>
Let's fit this model to data.

In [ ]:
ATLAS_fit_results = cabinetry.fit.fit(ATLAS_model, ATLAS_data)

<br>
To parse the results more easily, let's visualize them.

In [ ]:
cabinetry.visualize.pulls(ATLAS_fit_results)

<br>

# Bonus material

Additional material that probably does not fit into the main talk. Check it out for more information!

<br>

### Command line interface

`cabinetry` also provides a [command line interface](https://cabinetry.readthedocs.io/en/latest/cli.html). Feel free to explore it!

In [ ]:
! cabinetry --help

<br>

### Input histograms via a custom function

It is possible to inject custom code into `cabinetry`, which is used for template histogram creation. You can find more details in the [documentation](https://cabinetry.readthedocs.io/en/latest/advanced.html#overrides-for-template-building).

In [ ]:
my_router = cabinetry.route.Router()

# define a custom template builder function that is executed for data samples
@my_router.register_template_builder(sample_name="Data")
def build_data_hist(region: dict, sample: dict, systematic: dict, template: str):
    hist = bh.Histogram(
        bh.axis.Variable(region["Binning"], underflow=False, overflow=False),
        storage=bh.storage.Weight(),
    )
    yields = np.asarray([17, 12, 25, 20])
    variance = np.asarray([1.5, 1.2, 1.8, 1.6])
    hist[...] = np.stack([yields, variance], axis=-1)
    return hist  # return a boost-histogram histogram

custom_config = copy.deepcopy(config)
custom_config["General"]["HistogramFolder"] = "histograms_custom/"
cabinetry.templates.build(custom_config, router=my_router)

The histogram creation called our function to create the data histogram.

We can load the histogram and check the yields to verify they were correctly picked up.

In [ ]:
h = cabinetry.histo.Histogram.from_path(pathlib.Path("histograms_custom/Signal_region_Data.npz"))
h.values()

<br>

### Histogram smoothing

This shows how to apply smoothing to a histogram after producing it. We copy our config and save the resulting histograms in a new location to not interfere with the model we have used so far.

In [ ]:
smoothing_config = copy.deepcopy(config)
smoothing_config["General"]["HistogramFolder"] = "histograms_smoothing/"
smoothing_config.update({
   "Systematics":[
      {
         "Name":"Modeling",
         "Up": {"SamplePath": "prediction.root", "Tree": "background_varied"},
         "Down": {"Symmetrize": True},
         "Samples": "Background",
         "Smoothing": {"Algorithm": "353QH, twice"},  # smoothing applied
         "Type": "NormPlusShape"
      }
   ]
})
cabinetry.templates.build(smoothing_config, method="uproot")
cabinetry.templates.postprocess(smoothing_config)
_ = cabinetry.visualize.templates(smoothing_config)

The original histogram is shown with the dotted line, and the dashed line shows the histogram after applying the [353QH, twice](https://cds.cern.ch/record/186223/) algorithm (same as `TH1::SmoothArray` in `ROOT`).

<br>

### Exploring histograms with `hist`

`cabinetry` uses `boost-histogram` internally to handle histograms. That means we can use `hist` for visualization. There are multiple ways for loading histograms, the method below uses information from the config.

In [ ]:
cabinetry_histogram = cabinetry.histo.Histogram.from_config(
    config["General"]["HistogramFolder"],
    config["Regions"][0],  # we only have one region: Signal_region
    config["Samples"][0],  # data
    {"Name": "Nominal"},   # no systematics
)
_ = hist.Hist(cabinetry_histogram).plot()

<br>

### Going back and forth between `cabinetry` and `pyhf`

You can go back and forth between `cabinetry` and `pyhf`. Here is a simple example that builds the Asimov dataset with `cabinetry` and performs a fit with `pyhf`.

In [ ]:
asimov_data = cabinetry.model_utils.asimov_data(model)
pyhf.infer.mle.fit(asimov_data, model)

<br>

You can also take advantage of the `pyhf` workspace format and edit parameters directly and then build a new model. Let's fix the `Modeling` nuisance parameter to `-0.5` and repeat the fit, this time with `cabinetry`. The model is obtained directly via `pyhf` here instead of using `cabinetry.model_utils.model_and_data`.

In [ ]:
spec_edited = copy.deepcopy(spec)
spec_edited["measurements"][0]["config"]["parameters"].append({"name": "Modeling", "inits": [-0.5], "fixed": True})
model_edited = pyhf.Workspace(spec_edited).model()
_ = cabinetry.fit.fit(model_edited, data)

The `Modeling` parameter shows up at its fixed value `-0.5` in the fit results, with an uncertainty of `0` since it is set to constant.

<br>

### Further MLE options: MINOS and goodness-of-fit

The `cabinetry.fit.fit` API allows use of the [MINOS algorithm](https://iminuit.readthedocs.io/en/stable/reference.html#iminuit.Minuit.minos) to compute confidence intervals via parameter scans. All parameters for which the algorithm should be run have to be listed.

In [ ]:
_ = cabinetry.fit.fit(model, data, minos=["Signal_norm", "Modeling"])

MINOS uncertainties are not restricted to be symmetric by construction.

`cabinetry` also implementes a goodness-of-fit calculation using the [saturated model](http://www.physics.ucla.edu/~cousins/stats/cousins_saturated.pdf).

In [ ]:
_ = cabinetry.fit.fit(model, data, goodness_of_fit=True)

<br>

### Likelihood scan

We can perform a likelihood scan, where we hold a parameter fixed at different values and perform a MLE for the other parameters, recording the likelihood at each step in the scan.

In [ ]:
scan_results = cabinetry.fit.scan(model, data, "Signal_norm", n_steps=5)

<br>
The scan in this example shows relatively good agreement with a naive Gaussian approximation.

In [ ]:
cabinetry.visualize.scan(scan_results)

<br>

### Nuisance parameter ranking

We can rank nuisance parameters (NPs) by their impact on the POI: how much does the POI change if the NP varies within its uncertainty? This requires a lot of MLE fits.

In [ ]:
ranking_results = cabinetry.fit.ranking(model, data, fit_results=fit_results)

<br>
The figure visualizes the impact of nuisance parameters on the POI in order of decreasing impact.

In [ ]:
cabinetry.visualize.ranking(ranking_results)